In [5]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import curve_fit
import numpy as np
from datetime import datetime
from logistic_functions import *

In [6]:
# read in sales data, units are [NumberOfVehicles / Year], even though the values are reported monthly
vfacts_sales_df = pd.read_csv('../data/processed/vfacts_df.csv')
evc_sales_df = pd.read_csv('../data/processed/evc_df.csv')

# read in new registrations data, units are [NumberOfVehicles / Year]
nsw_new_regos_df = pd.read_csv('../data/processed/nsw_rego_data.csv')

# read in stock data, units are [NumberOfVehicles]
abs_stock_df = pd.read_csv('../data/processed/abs_df.csv')
bitre_stock_df = pd.read_csv('../data/processed/bitre_df.csv')
stock_df = abs_stock_df.drop(columns=["2021-01-31"]).merge(bitre_stock_df, how="outer")

Compare the national aggregation of ABS stock over time with the VFACTS sales over time (both in per month values)

In [7]:
# Extract the year in decimal form from the date to use as x-axis for plotting and fitting
def get_year_from_date(df):

    # check if string can be converted to pandas datetime format
    def is_date(string):
        
        if isinstance(string, pd.Timestamp):
            return True
        else:    
            try:
                datetime.strptime(string, '%Y-%m-%d')
                return True
            except ValueError:
                return False

    year = []
    for col in df.columns:
        if is_date(col):
            year.append(datetime.strptime(col, "%Y-%m-%d").year + datetime.strptime(col, "%Y-%m-%d").month / 12)

    return year


year_vfacts = get_year_from_date(vfacts_sales_df)
year_evc = get_year_from_date(evc_sales_df)
year_stock = get_year_from_date(stock_df)
year_nsw = get_year_from_date(nsw_new_regos_df) 

In [99]:
def get_calibrated_adoption_curve(tin, f, tout, calibration_points = [(2040,99,1)]):

    a0 = 0.5
    b0 = -4

    p_guess = [a0, b0]

    base_year = tin[0]

    t1 = np.array(tin)-base_year
    t2 = np.array(tout)-base_year
    f1 = np.array(f)
    sigma = np.ones(len(t1))

    for p in calibration_points:
        print(p)
        print(base_year)
        t1 = np.append(t1,p[0]-base_year)
        f1 = np.append(f1,p[1]/100)
        if len(p) == 3:
            print('Appending sigma with value: ', p[2])
            sigma = np.append(sigma,p[2])
        else:
            sigma = np.append(sigma,1)


    print(t1)
    print(f1)
    p2_opt = curve_fit(f=logistic, xdata=t1, ydata=f1, p0=p_guess, sigma=sigma)
    a2 = p2_opt[0][0]
    b2 = p2_opt[0][1]

    p_guess3 = [a2, b2, 0]
    p3_opt = curve_fit(f=elog, xdata=t1, ydata=f1, p0=p_guess3, sigma=sigma)
    a3 = p3_opt[0][0]
    b3 = p3_opt[0][1]
    u3 = p3_opt[0][2]

    return logistic(t2, a2, b2), elog(t2, a3, b3, u3)



Visualise the data and fit

just a note: Vehicle stock, scrapping rate ABS Catalogue No. 9309.0 - Motor Vehicle Census, Australia, 31 Jan 2021 (ABS, 2021a) 

In [100]:
stock_df.groupby(["State","Vehicle Type","Fuel Type"]).sum().drop(columns=["Postcode"])

2013-01-31  2014-01-31  2015-01-31  \
State Vehicle Type     Fuel Type                                             
ACT   Light Commercial Diesel              10184.0     11784.0     13565.0   
                       Dual fuel             987.0       950.0       832.0   
                       Electric                8.0        10.0        21.0   
                       Hybrid electric         0.0         0.0         0.0   
                       Other                 600.0       596.0       564.0   
...                                            ...         ...         ...   
WA    Passenger        Dual fuel              48.0        41.0        24.0   
                       Electric              354.0       408.0       425.0   
                       Hybrid electric         0.0         0.0         0.0   
                       Other               39119.0     36687.0     33196.0   
                       Petrol            1250111.0   1289730.0   1299734.0   

                                        2016-01-31  2017-01-31  2018-01-31  \
State Vehicle Type     Fuel Type                                             
ACT   Light Commercial Diesel              15343.0     17020.0     18958.0   
                       Dual fuel             729.0       680.0       573.0   
                       Electric               18.0        24.0        23.0   
                       Hybrid electric         0.0         0.0         0.0   
                       Other                 524.0       489.0       462.0   
...                                            ...         ...         ...   
WA    Passenger        Dual fuel              33.0        33.0        15.0   
                       Electric              495.0       505.0       578.0   
                       Hybrid electric         0.0         0.0         0.0   
                       Other               29560.0     26256.0     23145.0   
                       Petrol            1302539.0   1296638.0   1292414.0   

                                        2019-01-31  2020-01-31  2021-01-31  \
State Vehicle Type     Fuel Type                                             
ACT   Light Commercial Diesel              19990.0     21598.0     25036.0   
                       Dual fuel             442.0       350.0       369.0   
                       Electric               28.0        33.0         3.0   
                       Hybrid electric         0.0         0.0         3.0   
                       Other                 406.0       361.0       306.0   
...                                            ...         ...         ...   
WA    Passenger        Dual fuel              12.0        12.0     14683.0   
                       Electric              630.0      1039.0      1279.0   
                       Hybrid electric         0.0         0.0     19865.0   
                       Other               20146.0     18774.0      2510.0   
                       Petrol            1286611.0   1360771.0   1338415.0   

                                        2022-01-31  2023-01-31  
State Vehicle Type     Fuel Type                                
ACT   Light Commercial Diesel              26790.0     28282.0  
                       Dual fuel             315.0       286.0  
                       Electric                3.0         9.0  
                       Hybrid electric        12.0        50.0  
                       Other                 294.0       280.0  
...                                            ...         ...  
WA    Passenger        Dual fuel           13006.0     11646.0  
                       Electric             3267.0      6916.0  
                       Hybrid electric     27414.0     35888.0  
                       Other                2198.0      2028.0  
                       Petrol            1348980.0   1360502.0  

[95 rows x 11 columns]

In [101]:
evc_sales_df.set_index("Fuel Type").loc["Electric"]/evc_sales_df.set_index("Fuel Type").loc["All"]*100

2011-12-31    0.000000
2012-12-31    0.013676
2013-12-31    0.013038
2014-12-31    0.033676
2015-12-31    0.064286
2016-12-31    0.058554
2017-12-31    0.100490
2018-12-31    0.099788
2019-12-31    0.512027
2020-12-31    0.589522
2021-12-31    1.631810
2022-12-31    3.235203
2023-06-30    7.763658
dtype: float64

In [105]:
#def national_plot(fuel_type):
fuel_type = "Electric"

# ABS stock data
fueltype_stock_change_per_year = (
    stock_df.astype({"Postcode": "string"})
    .groupby(["Fuel Type"])
    .sum()
    .drop(columns=["State", "Postcode", "Vehicle Type"])
    .diff(axis=1)
)  # change to vehicle stock per year
total_stock_change_per_year = fueltype_stock_change_per_year.sum(axis=0)
fueltype_stock_change_per_year_percent = (
    fueltype_stock_change_per_year / total_stock_change_per_year * 100
)

# VFACTS sales data
fueltype_sales_per_year = vfacts_sales_df.groupby(["Fuel Type"]).sum().drop(columns=["State","Vehicle Type"]) 
total_sales_per_year = fueltype_sales_per_year.sum()
fueltype_sales_per_year_percent = (
    fueltype_sales_per_year / total_sales_per_year * 100
)

fig = make_subplots(rows=1,cols=2)

x1 = year_stock
y1 = fueltype_stock_change_per_year.loc[fuel_type].values.tolist()

# Add traces
fig.add_trace(
    go.Scatter(x=x1, y=y1, name="ABS Stock Change", opacity=0.5, line={"width": 5}),
    #secondary_y=False,
    row = 1, col = 1,
)

x2 = year_vfacts
y2 = fueltype_sales_per_year.loc[fuel_type].values.tolist()

fig.add_trace(
    go.Scatter(x=x2, y=y2, name="VFACTS Sales", opacity=0.5, line={"width": 5}),
    #secondary_y=False,
    row = 1, col = 1,
)

x3 = year_stock
y3 = fueltype_stock_change_per_year_percent.loc[fuel_type].values.tolist()

fig.add_trace(
    go.Scatter(x=x3, y=y3, name="ABS Stock Change Percent"),
    #secondary_y=True,
    row = 1, col = 2,
)

x4 = year_vfacts
y4 = fueltype_sales_per_year_percent.loc[fuel_type].values.tolist()

fig.add_trace(
    go.Scatter(x=x4, y=y4, name="VFACTS Sales Percent"),
    secondary_y=False,
    row = 1, col = 2,
)

three_par = True

x5 = np.linspace(2013, 2045)
calibration_points = [(2030, 60, 0.5), (2035, 99, 0.5)] # (year, percent, sigma) Note: sigma is optional and lower than 1 means more weight

y5, y6 = get_calibrated_adoption_curve(
    x4, [_y / 100 for _y in y4], x5, calibration_points=calibration_points
)
y5 = [_y * 100 for _y in y5]
y6 = [_y * 100 for _y in y6]

yfit = y5
if three_par:
    yfit = y6

fig.add_trace(
    go.Scatter(
        x=[f[0] for f in calibration_points],
        y=[f[1] for f in calibration_points],
        name="Calibration Points",
        marker=dict(size=8, color="blue", symbol="circle-open"),
        mode="markers",
    ),
    secondary_y=False,
    row = 1, col = 2,
)

fig.add_trace(
    go.Scatter(x=x5, y=yfit, name="Adoption Curve (VFACTS)", line={"dash": "dash"}),
    secondary_y=False,
    row = 1, col = 2,
)

x7 = year_evc
y7 = (evc_sales_df.set_index("Fuel Type").loc["Electric"]/evc_sales_df.set_index("Fuel Type").loc["All"]*100).tolist() 
fig.add_trace(
    go.Scatter(x=x7, y=y7, name="EVC Sales Percent"),
    secondary_y=False,
    row = 1, col = 2,
)


yfit2, yfit3 = get_calibrated_adoption_curve(
    x7, [_y / 100 for _y in y7], x5, calibration_points=calibration_points
)
yfit2 = [_y * 100 for _y in yfit2]
yfit3 = [_y * 100 for _y in yfit3]

yfit = yfit2
if three_par:
    yfit = yfit3

fig.add_trace(
    go.Scatter(x=x5, y=yfit, name="Adoption Curve (EVC)", line={"dash": "dash"}),
    secondary_y=False,
    row = 1, col = 2,
)

#x8 = year_nsw
#y8 = nsw_new_regos_df["NSW Percent EV (Total)"].tolist()
#fig.add_trace(
#    go.Scatter(x=x8, y=y8, name="NSW Percent EV (Total)"),
#    secondary_y=False,
#    row = 1, col = 2,
#)
#
#
#yfit2, yfit3 = get_calibrated_adoption_curve(
#    x8, [_y / 100 for _y in y8], x5, calibration_points=calibration_points
#)
#yfit2 = [_y * 100 for _y in yfit2]
#yfit3 = [_y * 100 for _y in yfit3]
#
#yfit = yfit2
#if three_par:
#    yfit = yfit3
#
#fig.add_trace(
#    go.Scatter(x=x5, y=yfit, name="Adoption Curve (NSW)", line={"dash": "dash"}),
#    secondary_y=False,
#    row = 1, col = 2,
#)

# Add figure title
fig.update_layout(
    title_text="Sales & Stock Change Data for Fuel Type = " + fuel_type
)

# Set x-axis title
fig.update_xaxes(title_text="Year")

# Set y-axes titles
fig.update_yaxes(title_text="Vehicles Per Year", secondary_y=False, row=1, col=1 )
fig.update_yaxes(title_text="Percent of All Fuel Types", secondary_y=False, row=1, col=2)

fig.show()
# download as html
fig.write_html("national_plot.html")

#national_plot("Electric")

(2030, 60, 0.5)
2020.25
Appending sigma with value:  0.5
(2035, 99, 0.5)
2020.25
Appending sigma with value:  0.5
[ 0.    0.25  0.5   0.75  1.    1.25  2.25  2.5   2.75  3.    3.25  3.5
  9.75 14.75]
[0.         0.         0.00197539 0.00200476 0.00379836 0.00405497
 0.01877767 0.02802933 0.03230922 0.06752083 0.07762017 0.07638282
 0.6        0.99      ]
(2030, 60, 0.5)
2012.0
Appending sigma with value:  0.5
(2035, 99, 0.5)
2012.0
Appending sigma with value:  0.5
[ 0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  11.5 18.
 23. ]
[0.00000000e+00 1.36758893e-04 1.30375427e-04 3.36762481e-04
 6.42857143e-04 5.85536888e-04 1.00490368e-03 9.97879061e-04
 5.12027389e-03 5.89521739e-03 1.63180982e-02 3.23520342e-02
 7.76365820e-02 6.00000000e-01 9.90000000e-01]


In [98]:
import numpy as np

a = np.array([x for x in range(1, 10)])
np.append(a,100)
a.append(100)


AttributeError: 'numpy.ndarray' object has no attribute 'append'

Notes to consider
- The 3 parameter model isn't really a useful 3rd parameter, in that any devation from 1 means non 100%, so let's look elsewhere. In fact, that c should likely be interpreted as something to use to force a saturation level if not 1, rather than let the model fit the value. 
- The logistic curve is 1 at +inf and 0 at ?, so specifying 1 as a calibration is likely is pushing th model to do strange things ... demonstrate with 0.9, 0.99, 0.999, 0.9999 and 1.0 at fixed time
- Perhaps we can see how good the model is by not using calibration points at all, and using the historical data to see where the logistical curve model forecasts. 
- Then, add in additional parameters representitive of 
    - Incentive scheme to meet sales target
    - Emissions standard
    - Accelerating uptake (rather than decelerating) as supply, etc forces adpoption.

In [23]:
x =  [_x-base_year for _x in x7]
y = [_y / 100 for _y in y7]
x

[2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 13.5]

In [ ]:
abs_df

In [ ]:
national_passenger_df = df

In [ ]:
national_df = df

In [ ]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots

def plot_sales(df):
    subplot_fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig1 = px.area(df, x='Date', y='Sales Per Month', color='Fuel Type')
    for i in range(len(fig1['data'])):
        fig1['data'][i]['line']['width']=0
    fig2 = px.line(df, x='Date', y='Percent Sales Per Month', color='Fuel Type')
    fig2.update_traces(yaxis='y2')
    subplot_fig.add_traces(fig1.data + fig2.data)
    subplot_fig.show()

plot_sales(national_df)
plot_sales(national_passenger_df)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=75de7644-8bd4-4ecc-bdb2-2c9ef0ed94e0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>